In [266]:
import pandas as pd
import numpy as np
import copy

In [72]:
class node:
#     nextLayerNodeCount=0
#     inputToNode=0
#     outputOfNode=0
#     bias=0
    def __init__(self,nextLayerNodeCount):
        self.outGoingWeights=[]
        self.initializeWeights(nextLayerNodeCount)
        self.nextLayerNodeCount = nextLayerNodeCount
        return
    
    def initializeWeights(self,count):
        for i in range(0,count):
            self.outGoingWeights.append(np.random.rand())
        return
    
    def getOutputOfNode(self,inputToNode):
        self.inputToNode=inputToNode
        self.outputOfNode = self.activationFunction(inputToNode)
        return self.outputOfNode
        
    def activationFunction(self,inputToNode):
        return 1/(1+np.exp(-inputToNode))
    
    def getOutGoingWeights(self):
        return self.outGoingWeights
    
    def updateOutGoingWeights(self,delta_W):
        for i in range(0,self.nextLayerNodeCount):
            self.outGoingWeights[i]=self.outGoingWeights[i]+delta_W
        return

In [335]:
class Layer:
#     numOfNodes = 0
#     nextLayerNodeCount=0
#     nodes=[]
#     layerType="hidden"
#     inputs=[]
#     outputs=[]
#     delta=[]
#     nextLayer
#     prevLayer
#     inputToNextLayer=[]
    
    def __init__(self,Layertype,nodeCount,nextLayerNodeCount,prevLayer):
        self.nextLayerNodeCount = nextLayerNodeCount
        self.nodes=[]
        self.delta=[]
        self.numOfNodes=nodeCount
        for i in range(0,nodeCount):
            self.nodes.append(node(nextLayerNodeCount))
        self.layerType =Layertype
        self.prevLayer = prevLayer
        return
    
    def setNextLayer(self,nextLayer):
        self.nextLayer = nextLayer
        return
    
    def doForwardPropagation(self,inputFromPrevLayer):
        outputFromCurrentLayer = copy.deepcopy(inputFromPrevLayer)
        self.inputs=inputFromPrevLayer
        if(self.layerType=="input"):
            self.outputs = inputFromPrevLayer
        if(self.layerType != "input"): #applying activation function to every layer except input layer
            for nodeInCurrentLayer in range(0,self.numOfNodes):
                outputFromCurrentLayer[nodeInCurrentLayer] = self.nodes[nodeInCurrentLayer].getOutputOfNode(inputFromPrevLayer[nodeInCurrentLayer])
            self.outputs=outputFromCurrentLayer
        self.nextLayerNodeCount = self.nodes[0].nextLayerNodeCount
        if(self.layerType != "output"): #calculating input for next layer
            weightMatrix = []
            inputForNextLayer = []
            for i in range(0,self.numOfNodes):
                weightMatrix.append(self.nodes[i].getOutGoingWeights())
            for nodeInNextLayer in range(0,self.nextLayerNodeCount):
                summation = 0
                for nodeInCurrentLayer in range(0,self.numOfNodes):
                    summation = summation + (weightMatrix[nodeInCurrentLayer][nodeInNextLayer]*outputFromCurrentLayer[nodeInCurrentLayer])
                inputForNextLayer.append(summation)
            self.inputToNextLayer=inputForNextLayer
            return inputForNextLayer
        self.outputs=outputFromCurrentLayer
        return outputFromCurrentLayer
    
    def calculateDeltaForLayer(self,error):
        if(self.layerType=="output"): #using formula delta_w = n*delta*outputOfThisNode and delta = y(1-y)*(y-y_cap)
            #initialize delta list
            if(len(self.delta)<1):
                for i in range(0,self.numOfNodes):
                    self.delta.append(0)
            for i in range(0,self.numOfNodes):
                self.delta[i] = self.outputs[i]*(1-self.outputs[i])*error
            return
        else:
            #initialize delta list
            if(len(self.delta)<1):
                for i in range(0,self.numOfNodes):
                    self.delta.append(0)
            nextLayerNodeCount = self.nextLayerNodeCount
            nextLayerDelta = self.nextLayer.delta
            for i in range(0,self.numOfNodes):
                currentNodeWeights = self.nodes[i].getOutGoingWeights()
                summation = 0
                for j in range(0,len(nextLayerDelta)):
                    summation = summation + currentNodeWeights[j]*nextLayerDelta[j]
                self.delta[i]=summation*self.outputs[i]*(1-self.outputs[i])
            return
    
    def doWeightUpdateForLayer(self,learningRate):
        #only called on input and hidden layers.
        for i in range(0,self.numOfNodes):
            delta_w=self.delta[i]*learningRate*self.outputs[i] #since the output of current layer will be input in the next layer
            self.nodes[i].updateOutGoingWeights(delta_w)
        return            

In [359]:
class NeuralNetwork :
#     inputLayer
#     outputLayer
#     noOfHiddenLayer
#     nodeCount in each layer
#     hiddenLayers
    
    def __init__(self,nodeCountInInputLayer,nodeCountInOutputLayer,nodeCountInHiddenLayers):
        self.nodeCountInInputLayer = nodeCountInInputLayer
        self.nodeCountInOutputLayer = nodeCountInOutputLayer
        self.nodeCountInHiddenLayers = nodeCountInHiddenLayers
        self.hiddenLayers=[]
        self.initializeLayer(nodeCountInInputLayer,"input")
        self.initializeLayer(nodeCountInHiddenLayers,"hidden")
        self.initializeLayer(nodeCountInOutputLayer,"output")
        return
    
    def initializeLayer(self,nodeCountInLayer,layerType):
        if(layerType=="input"):
            self.inputLayer = Layer(layerType,nodeCountInLayer,self.nodeCountInHiddenLayers[0],None)
            return self.inputLayer
        if(layerType=="hidden"):
            prevLayer = self.inputLayer
            self.noOfHiddenLayer = len(nodeCountInLayer)
            for i in range(0,self.noOfHiddenLayer-1):
                self.hiddenLayers.append(Layer(layerType,nodeCountInLayer[i],nodeCountInLayer[i+1],prevLayer))
                prevLayer = self.hiddenLayers[i-1]
            #now manually adding the last hidden layer, since last wont be added by the above loop.
            if(self.noOfHiddenLayer > 1):
                prevLayer = self.hiddenLayers[self.noOfHiddenLayer-2]
                self.hiddenLayers.append(Layer(layerType,nodeCountInLayer[self.noOfHiddenLayer-1],self.nodeCountInOutputLayer,prevLayer))
            else :
                #handling the case that if only one hidden layer is there
                self.hiddenLayers.append(Layer(layerType,nodeCountInLayer[0],self.nodeCountInOutputLayer,prevLayer))
            #setting next layer info for future use
            self.inputLayer.setNextLayer(self.hiddenLayers[0])
            return
        if(layerType=="output"):
            self.outputLayer = Layer(layerType,self.nodeCountInOutputLayer,0,self.hiddenLayers[self.noOfHiddenLayer-1])
            #setting next layer info for future use
            self.outputLayer.setNextLayer(None)
            for i in range(0,self.noOfHiddenLayer-1):
                self.hiddenLayers[i].setNextLayer(self.hiddenLayers[i+1])
            self.hiddenLayers[self.noOfHiddenLayer-1].setNextLayer(self.outputLayer)
            return
    def doForwardPropagation(self,inputs):
        inputToNextLayer = self.inputLayer.doForwardPropagation(inputs)
        outputOfPrevLayer = inputToNextLayer
        for i in range(0,self.noOfHiddenLayer):
            inputToNextLayer = self.hiddenLayers[i].doForwardPropagation(outputOfPrevLayer)
            outputOfPrevLayer = inputToNextLayer
        output = self.outputLayer.doForwardPropagation(inputToNextLayer)
        return output
    
    def doBackwardPropagation(self,predictedOutput,desiredOutput,learningRate):
        error = desiredOutput - predictedOutput
        #all the weight updates will take place after calculation of all the deltas for all the layers.
        self.outputLayer.calculateDeltaForLayer(error)
        for i in range(len(self.hiddenLayers)-1,-1,-1):
            self.hiddenLayers[i].calculateDeltaForLayer(error)
        self.inputLayer.calculateDeltaForLayer(error)
        #now updating weights for each
        for i in range(len(self.hiddenLayers)-1,-1,-1):
            self.hiddenLayers[i].doWeightUpdateForLayer(learningRate)
        self.inputLayer.doWeightUpdateForLayer(error)
        return

In [374]:
nn = NeuralNetwork(2,1,[3,2])

In [375]:
print((nn.inputLayer.nodes[0].outGoingWeights))
print((nn.inputLayer.nodes[1].outGoingWeights))
# print((nn.inputLayer.nodes[4].outGoingWeights))
print((nn.hiddenLayers[0].nodes[0].outGoingWeights))
print((nn.hiddenLayers[0].nodes[1].outGoingWeights))
print((nn.outputLayer.nodes[0].outGoingWeights))

[0.05518640190427915, 0.27208934402039786, 0.9119389699029733]
[0.02438606604502125, 0.9654264760793342, 0.3177448240490972]
[0.3893008818646333, 0.13978766379808993]
[0.2985889242225881, 0.15400675718551404]
[]


In [376]:
print(nn.inputLayer.doForwardPropagation([0.35,0.35]))
print(nn.inputLayer.inputs)
print(nn.inputLayer.outputs)
#the values are correct
#thus forward propagation works for i/p, hidden, o/p layers

[0.02785036378225514, 0.43313053703490617, 0.43038932788322465]
[0.35, 0.35]
[0.35, 0.35]


In [377]:
predOutput = nn.doForwardPropagation([0.35,0.35])
print(predOutput)
print("error is equal to : "+str(predOutput[0] - 0.5))
#thus forward propagation is working well

[0.6946646406637885]
error is equal to : 0.19466464066378852


In [378]:
print(nn.doBackwardPropagation(predOutput[0],0.5,0.5))

None


In [379]:
for i in range(0,10000):
    predOutput = nn.doForwardPropagation([0.35,0.35])
    nn.doBackwardPropagation(predOutput[0],0.5,0.5)

In [380]:
predOutput = nn.doForwardPropagation([0.35,0.35])
print(predOutput)
print("error is equal to : "+str(predOutput[0] - 0.5))

[0.5053998457790096]
error is equal to : 0.005399845779009649
